# Azure AI Search s integráciou NVIDIA NIM a LlamaIndex

V tomto notebooku si ukážeme, ako využiť AI modely od NVIDIA a LlamaIndex na vytvorenie výkonného Retrieval-Augmented Generation (RAG) pipeline. Použijeme veľké jazykové modely (LLMs) a embeddingy od NVIDIA, integrujeme ich s Azure AI Search ako vektorovým úložiskom a vykonáme RAG na zlepšenie kvality a efektivity vyhľadávania.

## Výhody
- **Škálovateľnosť**: Využite veľké jazykové modely od NVIDIA a Azure AI Search na škálovateľné a efektívne vyhľadávanie.
- **Nákladová efektívnosť**: Optimalizujte vyhľadávanie a získavanie informácií pomocou efektívneho vektorového úložiska a hybridných vyhľadávacích techník.
- **Vysoký výkon**: Kombinujte výkonné LLMs s vektorovým vyhľadávaním pre rýchlejšie a presnejšie odpovede.
- **Kvalita**: Zachovajte vysokú kvalitu vyhľadávania tým, že odpovede LLMs budú založené na relevantných získaných dokumentoch.

## Predpoklady
- 🐍 Python 3.9 alebo vyšší
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API kľúč na prístup k LLMs a embeddingom od NVIDIA prostredníctvom mikroservisov NVIDIA NIM

## Pokryté funkcie
- ✅ Integrácia NVIDIA LLM (použijeme [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddingy od NVIDIA (použijeme [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Pokročilé režimy získavania informácií v Azure AI Search
- ✅ Indexovanie dokumentov pomocou LlamaIndex
- ✅ RAG pomocou Azure AI Search a LlamaIndex s LLMs od NVIDIA

Poďme na to!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Inštalácia a požiadavky
Vytvorte prostredie Python s použitím verzie Pythonu >3.10.

## Začíname!


Na začiatok budete potrebovať `NVIDIA_API_KEY` na používanie modelov NVIDIA AI Foundation:
1) Vytvorte si bezplatný účet na [NVIDIA](https://build.nvidia.com/explore/discover).
2) Kliknite na model, ktorý si vyberiete.
3) V časti Input vyberte kartu Python a kliknite na **Get API Key**, potom kliknite na **Generate Key**.
4) Skopírujte a uložte vygenerovaný kľúč ako NVIDIA_API_KEY. Odtiaľ budete mať prístup k endpointom.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Príklad RAG s použitím LLM a vkladania
### 1) Inicializácia LLM
`llama-index-llms-nvidia`, známy aj ako NVIDIA LLM konektor, umožňuje pripojenie a generovanie z kompatibilných modelov dostupných v katalógu API od NVIDIA. Zoznam modelov na dokončenie chatov nájdete tu: https://build.nvidia.com/search?term=Text-to-Text

Tu použijeme **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inicializácia vkladania
`llama-index-embeddings-nvidia`, známy aj ako NVIDIA's Embeddings connector, vám umožňuje pripojiť sa k kompatibilným modelom dostupným v katalógu NVIDIA API a generovať z nich. Vybrali sme model `nvidia/nv-embedqa-e5-v5` ako model na vkladanie. Zoznam modelov na textové vkladanie nájdete tu: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Vytvorte dotazovací nástroj na kladenie otázok o vašich údajoch

Tu je dotaz využívajúci čisté vektorové vyhľadávanie v Azure AI Search a ukotvenie odpovede na náš LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Tu je dopyt využívajúci hybridné vyhľadávanie v Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analýza vyhľadávania vektorov
Odpoveď LLM presne zachytáva kľúčové ekonomické dôsledky uvedené v zdrojovom texte týkajúce sa ruského akciového trhu. Konkrétne uvádza, že ruský akciový trh zaznamenal výrazný pokles, pričom stratil 40 % svojej hodnoty, a obchodovanie bolo pozastavené kvôli prebiehajúcej situácii. Táto odpoveď dobre korešponduje s informáciami uvedenými v zdroji, čo naznačuje, že LLM správne identifikoval a zhrnul relevantné detaily týkajúce sa dopadu na akciový trh v dôsledku krokov Ruska a uvalených sankcií.

#### Komentár k zdrojovým uzlom
Zdrojové uzly poskytujú podrobný prehľad ekonomických dôsledkov, ktorým Rusko čelilo kvôli medzinárodným sankciám. Text zdôrazňuje, že ruský akciový trh stratil 40 % svojej hodnoty a obchodovanie bolo pozastavené. Okrem toho spomína ďalšie ekonomické dopady, ako napríklad devalváciu rubľa a širšiu izoláciu ruskej ekonomiky. Odpoveď LLM účinne zhrnula kľúčové body z týchto uzlov, pričom sa zamerala na dopad na akciový trh, ako to bolo požadované v dotaze.


Pozrime sa na príklad, kde Hybridné vyhľadávanie neposkytuje dobre podloženú odpoveď:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybridné vyhľadávanie: Analýza odpovede LLM
Odpoveď LLM v príklade hybridného vyhľadávania naznačuje, že poskytnutý kontext neuvádza presný dátum invázie Ruska na Ukrajinu. Táto odpoveď naznačuje, že LLM využíva informácie dostupné v zdrojových dokumentoch, ale zároveň uznáva absenciu presných detailov v texte.

Odpoveď je presná v identifikácii, že kontext spomína udalosti súvisiace s ruskou agresiou, ale neuvádza konkrétny dátum invázie. To poukazuje na schopnosť LLM pochopiť poskytnuté informácie a zároveň rozpoznať medzery v obsahu. LLM efektívne nabáda používateľa, aby vyhľadal externé zdroje alebo historické záznamy na zistenie presného dátumu, čím prejavuje určitú opatrnosť, keď sú informácie neúplné.

### Analýza zdrojových uzlov
Zdrojové uzly v príklade hybridného vyhľadávania obsahujú úryvky z prejavu, ktorý sa zaoberá reakciou USA na kroky Ruska na Ukrajine. Tieto uzly zdôrazňujú širší geopolitický dopad a kroky, ktoré podnikli USA a ich spojenci v reakcii na inváziu, avšak neuvádzajú konkrétny dátum invázie. To je v súlade s odpoveďou LLM, ktorá správne identifikuje, že kontext neobsahuje presné informácie o dátume.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: Analýza odpovede LLM
V príklade Hybrid w/Reranking odpoveď LLM poskytuje dodatočný kontext tým, že uvádza, že udalosť sa odohrala šesť dní pred prednesením prejavu. To naznačuje, že LLM dokáže odvodiť dátum invázie na základe načasovania prejavu, aj keď na presnosť stále potrebuje poznať presný dátum prejavu.

Táto odpoveď ukazuje zlepšenú schopnosť využívať kontextové stopy na poskytnutie informatívnejšej odpovede. Zdôrazňuje výhodu preradenia (reranking), pri ktorom má LLM prístup k relevantnejším informáciám a dokáže ich uprednostniť, aby poskytlo presnejší odhad požadovaného detailu (t. j. dátumu invázie).

### Analýza zdrojových uzlov
Zdrojové uzly v tomto príklade zahŕňajú odkazy na načasovanie ruskej invázie, konkrétne zmienku, že sa odohrala šesť dní pred prejavom. Aj keď presný dátum stále nie je výslovne uvedený, uzly poskytujú časový kontext, ktorý umožňuje LLM poskytnúť prepracovanejšiu odpoveď. Zahrnutie tohto detailu ukazuje, ako môže preradenie zlepšiť schopnosť LLM extrahovať a odvodiť informácie z poskytnutého kontextu, čo vedie k presnejšej a informatívnejšej odpovedi.


**Poznámka:**
V tomto notebooku sme použili mikroservisy NVIDIA NIM z katalógu API od spoločnosti NVIDIA.  
Vyššie uvedené API, `NVIDIA (llms)`, `NVIDIAEmbedding` a [Azure AI Search Semantic Hybrid Retrieval (zabudované prehodnocovanie)](https://learn.microsoft.com/azure/search/semantic-search-overview). Upozorňujeme, že vyššie uvedené API môžu tiež podporovať mikroservisy hostované na vlastných serveroch.

**Príklad:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
